In [ ]:
# Install required libraries
!pip install transformers datasets gdown

In [ ]:
!pip install rust
!pip install tokenizers

In [ ]:
# Clone your GitHub repository for the emotion recognition project
!git clone https://github.com/Vincent-ZHQ/CA-MSER.git

# Install the project dependencies
!pip install -r CA-MSER/requirements.txt

In [3]:
# Download Wav2Vec2.0 pretrained model from Hugging Face
from transformers import Wav2Vec2Model

In [4]:
import torch
print("Is CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
features_file = '/content/drive/MyDrive/path/to/reduced_file.pkl'

Is CUDA available: True
CUDA version: 11.8


In [1]:
import sys
import os
import time
import json  # Import json for saving results


# Add the path to the directory containing train_ser.py to the Python path
sys.path.append('CA-MSER')  # Change the path if train_ser.py is in a different directory

import train_ser
from train_ser import parse_arguments
import sys
import pickle
import os
import time


repeat_kfold = 2 # to  perform 10-fold for n-times with different seed
localtime = time.localtime(time.time())
str_time = f'{str(localtime.tm_year)}-{str(localtime.tm_mon)}-{str(localtime.tm_mday)}-{str(localtime.tm_hour)}-{str(localtime.tm_min)}'

#------------PARAMETERS---------------#
force_download=True
features_file = 'IEMOCAP_multi.pkl'

#  leave-one-speaker-out
val_id = ['1M','1F','2M','2F','3M','3F','4M','4F','5M','5F']
test_id = ['1M','1F','2M','2F','3M','3F','4M','4F','5M','5F']

#  leave-one-session-out
# val_id = ['1M','2M','3M','4M','5M']
# test_id = ['1F','2F','3F','4F','5F']

num_epochs  = '30'
early_stop = '8'
batch_size  = '32'
lr          = '0.00001'
random_seed = 111
gpu = '1'
gpu_ids = ['0']
save_label = str_time#'0930_01'#'alexnet_pm_0704'


#Start Cross Validation
all_stat = []

for repeat in range(repeat_kfold):

    random_seed +=  (repeat*100)
    seed = str(random_seed)

    for v_id, t_id in list(zip(val_id, test_id)):

        train_ser.sys.argv      = [

                                  'train_ser.py',
                                  features_file,
                                  '--repeat_idx', str(repeat),
                                  '--val_id',v_id,
                                  '--test_id', t_id,
                                  '--gpu', gpu,
                                  '--gpu_ids', gpu_ids,
                                  '--num_epochs', num_epochs,
                                  '--early_stop', early_stop,
                                  '--batch_size', batch_size,
                                  '--lr', lr,
                                  '--seed', seed,
                                  '--save_label', save_label,#,
                                  '--pretrained'
                                  ]


        stat = train_ser.main(parse_arguments(sys.argv[1:]))
        all_stat.append(stat)
        os.remove(save_label+'.pth')

    # with open('allstat_iemocap_'+save_label+'_'+str(repeat)+'.pkl', "wb") as fout:
    #     pickle.dump(all_stat, fout)

n_total = repeat_kfold*len(val_id)
total_best_epoch, total_epoch, total_loss, total_wa, total_ua = 0, 0, 0, 0, 0

for i in range(n_total):
    print(i, ": ", all_stat[i][0], all_stat[i][1], all_stat[i][8], all_stat[i][9], all_stat[i][10])
    total_best_epoch += all_stat[i][0]
    total_epoch += all_stat[i][1]
    total_loss += float(all_stat[i][8])
    total_wa += float(all_stat[i][9])
    total_ua += float(all_stat[i][10])

print("AVERAGE:", total_best_epoch/n_total, total_epoch/n_total, total_loss/n_total, total_wa/n_total, total_ua/n_total )

print(all_stat)

****************************************

PARAMETERS:

****************************************


       ser_task: SLM
     repeat_idx: 0
         val_id: 1M
        test_id: 1M
     num_epochs: 30
     early_stop: 8
     batch_size: 32
             lr: 1e-05
    random_seed: 111
        use_gpu: 1
        gpu_ids: ['0']
     save_label: 2024-10-21-19-33
   oversampling: False
     pretrained: True
****************************************



Dataset normalized with minmax scaler
	Range before normalization: [-80.0, 3.8146973e-06]
	Range after  normalization: [0.0, 1.0]

<<DATASET>>

Val. speaker id : 1M
Test speaker id : 1M
Train data      : (9815, 1, 200, 300)
Train labels    : (9815,)
Eval. data      : (1187, 1, 200, 300)
Eval. label     : (557,)
Eval. seg labels: (1187,)
Eval. num seg   : (557,)
Test data       : (1187, 1, 200, 300)
Test label      : (557,)
Test seg labels : (1187,)
Test num seg    : (557,)


pytorch version:  2.1.1+cu118
cuda version:  11.8
cudnn version:  8700
gpu

/home/stud1/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/stud1/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



<< SER AlexNet Finetuning model initialized >>



/home/stud1/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Ser_Model(
  (alexnet_model): SER_AlexNet(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (4): ReLU(inplace=True)
      (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace=True)
      (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): ReLU(inplace=True)
      (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
    (classifier): Sequential(
      (0): Dropout(p=0.5, inplace=Fa

...

 ang  sad  hap  neu
  59    0    8    3
   0   54    9   21
  12    3  131   15
  21   19   82  105
5 6
Epoch 7  (lr = 1e-05)        Loss: 0.8035 - 0.9761 - WA: 64.39% <63.28%> - UA: 69.05% <72.46%>


100%|██████████| 19/19 [00:02<00:00,  6.45it/s]


 ang  sad  hap  neu
  65    0    1    4
   1   64    2   17
  24    7  103   27
  37   25   47  118
True
7 7
Epoch 8  (lr = 1e-05)        Loss: 0.7693 - 1.0287 - WA: 64.58% <64.58%> - UA: 71.25% <71.25%>


 86%|████████▌ | 264/308 [01:02<00:10,  4.20it/s]


KeyboardInterrupt: 

In [4]:
import requests
response = requests.get('http://huggingface.co/facebook/wav2vec2-base-960h/resolve/main/config.json', verify=True)

In [7]:
!pip install fairseq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 5.7 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Cython-3.0.11-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.2 kB)
Requested omegaconf<2.1 from https://files.pythonhosted.org/packages/d0/eb/9d63ce09dd8aa85767c65668d5414958ea29648a0eec80a4a7d311ec2684/omegaconf-2.0.6-py3-none-any.whl (from fairseq) has invalid metadata: .* suffix can only be used with `==` or `!=` operators
    PyYAML (>=5.1.*)
            ~~~~~~^
Please use pip<24.1 if you need to use this version.
Requested omegaconf<2.1 from https://files.pythonhosted.org/packages/e5/f6/043b6d255dd6fbf2025110cea35b87f4c5100a181681d8eab496269f0d5b/omegaconf-2.0.5-py3-none-any.whl (from fairseq) has invalid metadata: .* suffix can only be used with `==` or `!=` operators
    PyYAML (>